In [1]:
import torch
import os
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, random_split
from torchvision import datasets, transforms, models
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor
import matplotlib.pyplot as plt
from torchvision.io import read_image
from torch.utils.tensorboard import SummaryWriter

In [2]:
num_to_label= {
    0: "backpack",
    1: "book",
    2: "car",
    3: "pizza",
    4: "sandwich",
    5: "snake",
    6: "sock",
    7: "tiger",
    8: "tree",
    9: "watermelon"
}

label_to_num = dict((v,k) for k,v in num_to_label.items())

def obtain_labels(root_dir):
    labels = []

    ok = os.walk(root_dir, topdown=False)
    for root, _, files in ok:
        if root_dir == root: continue
        root = os.path.basename(root)
        labels += list(map(lambda x: [x, root], files))

    return np.array(labels)

In [3]:
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_labels = obtain_labels(img_dir)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels[idx, 1], self.img_labels[idx, 0])
        image = Image.open(img_path)
        label = self.img_labels[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    
dataset_transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

class LabelToNum(torch.nn.Module):
    def forward(self, label):
        return label_to_num[label]

label_transform = LabelToNum()

In [4]:
rn34_model = models.resnet34()
num_ftrs = rn34_model.fc.in_features
rn34_model.fc = torch.nn.Linear(num_ftrs, 10)

In [5]:
def split_valid(dataset, validation_split=0.2, random_seed=42):
    # Creating data indices for training and validation splits:
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)
    return train_sampler, valid_sampler

In [6]:
## Obtain Real datasets and split for validation sets.
train_dataset_real = CustomImageDataset(img_dir="data/real_train", transform=dataset_transform, target_transform=label_transform)
test_dataset_real = CustomImageDataset(img_dir="data/real_test", transform=dataset_transform, target_transform=label_transform)

## Obtain Sketch datasets and split for validation sets.
train_dataset_sketch = CustomImageDataset(img_dir="data/sketch_train", transform=dataset_transform, target_transform=label_transform)
test_dataset_sketch = CustomImageDataset(img_dir="data/sketch_test", transform=dataset_transform, target_transform=label_transform)

t_real_sample, v_real_sample = split_valid(train_dataset_real)
t_sketch_sample, v_sketch_sample = split_valid(train_dataset_sketch)

In [7]:
def train(src_loader, model, loss_fn, optimizer, epoch, device):
    running_loss = 0
    train_loss = 0
   
    for idx, (data, targets) in enumerate(src_loader):

        data = data.to(device)
        targets = targets.to(device)

        scores = model(data)
        loss = loss_fn(scores, targets)
        
        optimizer.zero_grad()
        loss.backward()

        # update the parameters according to gradients
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        train_loss += loss.item()

        # report every 100 iterations
        if idx % 25 == 24:
            print(' epoch {} loss: {:.4f}'.format(epoch+1, running_loss / 25))
            running_loss = 0.0
    
    train_loss = train_loss/(idx+1)

    print(f'Epoch [{epoch+1}], '
          f'Train Loss: {train_loss:.4f}, ')
    
    return train_loss


def test(src_loader, model, loss_fn, device):

    valid_loss = 0
    num_correct = 0
    num_samples = 0

    with torch.no_grad():
        for data, targets in src_loader:
            data = data.to(device)
            targets = targets.to(device)

            scores = model(data)['fc']
            _, predictions = scores.max(1)
            num_correct += (predictions == targets).sum()
            num_samples += predictions.size(0)

            valid_loss += loss_fn(scores, targets)
        
        print(f'{num_correct}/{num_samples} accuracy {num_correct/num_samples}')

    # Gather data and report
    valid_loss /= len(src_loader)
    accuracy = num_correct/num_samples
    print("Test Error: \n   Accuracy: {:.2f}, Avg loss: {:.4f} \n".format(100*accuracy, valid_loss))
    
    return valid_loss, accuracy

In [8]:
# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Training init
learning_rate = 0.001
epochs = 100
batch_size = 64

In [9]:
# Real DataLoaders
dl_train_real = DataLoader(
    train_dataset_real,
    batch_size=batch_size,
    sampler=t_real_sample,
    drop_last=True
)

dl_valid_real = DataLoader(
    train_dataset_real,
    batch_size=batch_size,
    sampler=v_real_sample,
    drop_last=False
)

dl_test_real = DataLoader(
    test_dataset_real,
    batch_size=batch_size,
    shuffle=True
)

## Sketch DataLoaders
dl_train_sketch = DataLoader(
    train_dataset_sketch,
    batch_size=batch_size,
    sampler=t_sketch_sample,
    drop_last=True
)

dl_valid_sketch = DataLoader(
    train_dataset_sketch,
    batch_size=batch_size,
    sampler=v_sketch_sample,
    drop_last=False
)

dl_test_sketch = DataLoader(
    test_dataset_sketch,
    batch_size=batch_size,
    shuffle=True
)

In [10]:
print(device)

cuda


In [11]:
## Optimizer construction using Adam.
params = [p for p in rn34_model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(
    params,
    lr=learning_rate,
)
rn34_model.to(device)
loss_fn = torch.nn.CrossEntropyLoss()

# ******************* TensorBoard *******************
writer = SummaryWriter(comment="ass2")

# ******************* Optimization *******************
best_accuracy = 0
for epoch in range(epochs):
    # train loop
    # set the module in training mode.
    rn34_model.train()
    train_loss = train(dl_train_sketch, rn34_model, loss_fn, optimizer, epoch, device)
    # save to tensorboard
    writer.add_scalar('train_loss', train_loss, epoch)

    # save model weights
    save_path = 'ckpt_{:04d}_22.pth'.format(epoch+1)
    torch.save(rn34_model.state_dict(), save_path)

    # validation loop
    # set the module in evaluation mode.
    rn34_model.eval()
    valid_loss, valid_accuracy = test(dl_valid_sketch, rn34_model, loss_fn, device)
    # save to tensorboard
    writer.add_scalar('valid_loss', valid_loss, epoch)
    writer.add_scalar('valid_accuracy', valid_accuracy, epoch)

    if valid_accuracy > best_accuracy:    # save the model with best validation accuracy
        save_path = 'ckpt_best_22.pth'
        torch.save(rn34_model.state_dict(), save_path)
        best_accuracy = valid_accuracy
    
    print(f'Epoch: {epoch+1}', f'Train_loss: {train_loss}', f'Valid_loss: {valid_loss}', f'Valid_acc: {valid_accuracy}')

writer.close()
print("Finished Training.")

KeyboardInterrupt: 

In [12]:
coral_loss_weight = 10
epochs = 20
learning_rate = 1e-2

In [13]:
class CORAL(torch.nn.Module):
    def forward(self, source, target):
        d = source.size(1)
        source_cov = torch.cov(source.t(), correction=1)
        target_cov = torch.cov(target.t(), correction=1)
        loss = torch.square(torch.norm(source_cov-target_cov, p='fro'))
        loss = loss / (4*d*d)
        return loss

In [14]:
def train_coral(src_loader, tgt_loader, model, loss_fn, optimizer, epoch, device):
    running_loss = 0
    train_loss = 0
    
    train_steps = min(len(src_loader), len(tgt_loader))

    coral = CORAL()

    for idx in range(train_steps):
        src_input, src_label = next(iter(src_loader))
        tgt_input, _ = next(iter(tgt_loader))

        src_input = src_input.to(device)
        src_label = src_label.to(device)
        tgt_input = tgt_input.to(device)

        src_outputs = model(src_input)
        tgt_outputs = model(tgt_input)

        loss = loss_fn(src_outputs['fc'], src_label)

        loss_coral = coral(src_outputs['flatten'], tgt_outputs['flatten'])

        joint_loss = loss + coral_loss_weight * loss_coral

        optimizer.zero_grad()
        joint_loss.backward()
        # update the parameters according to gradients
        optimizer.step()

        # Gather data and report
        running_loss += joint_loss.item()
        train_loss += joint_loss.item()

        # report every 100 iterations
        if idx % 25 == 24:
            print(' epoch {} loss: {:.4f}'.format(epoch+1, running_loss / 100))
            running_loss = 0.0
    
    train_loss = train_loss/train_steps

    print(f'Epoch [{epoch+1}], '
          f'Train Loss: {train_loss:.4f}, ')
    
    return train_loss

In [15]:
# MaskRCNN requires a backbone with an attached FPN
class Resnet34(torch.nn.Module):
    def __init__(self, return_nodes, num_classes):
        super(Resnet34, self).__init__()
        # Get a resnet34 backbone
        m = models.resnet34()
        train_nodes, eval_nodes = get_graph_node_names(m)
        print(eval_nodes)
        m.fc = torch.nn.Linear(m.fc.in_features, num_classes)
        # Extract 4 main layers
        self.body = create_feature_extractor(
            m, return_nodes=return_nodes)
        
    def forward(self, x):
        x = self.body(x)
        return x

In [16]:
return_nodes = {
    'layer4':'layer4',
    'flatten':'flatten',
    'fc':'fc'
}
rn34_model = Resnet34(return_nodes, 10)

['x', 'conv1', 'bn1', 'relu', 'maxpool', 'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu', 'layer1.0.conv2', 'layer1.0.bn2', 'layer1.0.add', 'layer1.0.relu_1', 'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu', 'layer1.1.conv2', 'layer1.1.bn2', 'layer1.1.add', 'layer1.1.relu_1', 'layer1.2.conv1', 'layer1.2.bn1', 'layer1.2.relu', 'layer1.2.conv2', 'layer1.2.bn2', 'layer1.2.add', 'layer1.2.relu_1', 'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.downsample.0', 'layer2.0.downsample.1', 'layer2.0.add', 'layer2.0.relu_1', 'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.relu', 'layer2.1.conv2', 'layer2.1.bn2', 'layer2.1.add', 'layer2.1.relu_1', 'layer2.2.conv1', 'layer2.2.bn1', 'layer2.2.relu', 'layer2.2.conv2', 'layer2.2.bn2', 'layer2.2.add', 'layer2.2.relu_1', 'layer2.3.conv1', 'layer2.3.bn1', 'layer2.3.relu', 'layer2.3.conv2', 'layer2.3.bn2', 'layer2.3.add', 'layer2.3.relu_1', 'layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.relu', 'layer3.0.conv2', 

In [17]:
## Optimizer construction using SGD
params = [p for p in rn34_model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=learning_rate,
    weight_decay=5e-4,
    momentum=0.9
)
rn34_model.to(device)
loss_fn = torch.nn.CrossEntropyLoss()

# ******************* TensorBoard *******************
writer = SummaryWriter(comment="ass2")

# ******************* Optimization *******************
best_accuracy = 0
for epoch in range(epochs):
    # train loop
    # set the module in training mode.
    rn34_model.train()
    train_loss = train_coral(dl_train_real, dl_train_sketch, rn34_model, loss_fn, optimizer, epoch, device)
    # save to tensorboard
    writer.add_scalar('train_loss', train_loss, epoch)

    # save model weights
    save_path = 'ckpt_{:04d}_22.pth'.format(epoch+1)
    torch.save(rn34_model.state_dict(), save_path)

    # validation loop
    # set the module in evaluation mode.
    rn34_model.eval()
    valid_loss, valid_accuracy = test(dl_valid_sketch, rn34_model, loss_fn, device)
    # save to tensorboard
    writer.add_scalar('valid_loss', valid_loss, epoch)
    writer.add_scalar('valid_accuracy', valid_accuracy, epoch)

    if valid_accuracy > best_accuracy:    # save the model with best validation accuracy
        save_path = 'ckpt_best_22.pth'
        torch.save(rn34_model.state_dict(), save_path)
        best_accuracy = valid_accuracy
    
    print(f'Epoch: {epoch+1}', f'Train_loss: {train_loss}', f'Valid_loss: {valid_loss}', f'Valid_acc: {valid_accuracy}')

writer.close()
print("Finished Training.")

Epoch [1], Train Loss: 2.3705, 
37/391 accuracy 0.09462915360927582
Test Error: 
   Accuracy: 9.46, Avg loss: 2.3724 

Epoch: 1 Train_loss: 2.370479871829351 Valid_loss: 2.372389554977417 Valid_acc: 0.09462915360927582
Epoch [2], Train Loss: 2.1314, 
60/391 accuracy 0.15345267951488495
Test Error: 
   Accuracy: 15.35, Avg loss: 2.4444 

Epoch: 2 Train_loss: 2.1314078072706857 Valid_loss: 2.4444022178649902 Valid_acc: 0.15345267951488495
Epoch [3], Train Loss: 1.9126, 
44/391 accuracy 0.11253196746110916
Test Error: 
   Accuracy: 11.25, Avg loss: 2.7861 

Epoch: 3 Train_loss: 1.9125944872697194 Valid_loss: 2.786140203475952 Valid_acc: 0.11253196746110916
Epoch [4], Train Loss: 1.6980, 
57/391 accuracy 0.1457800567150116
Test Error: 
   Accuracy: 14.58, Avg loss: 2.8406 

Epoch: 4 Train_loss: 1.6980008085568745 Valid_loss: 2.8406260013580322 Valid_acc: 0.1457800567150116
Epoch [5], Train Loss: 1.4956, 
52/391 accuracy 0.13299232721328735
Test Error: 
   Accuracy: 13.30, Avg loss: 2.9714 